# Process Mining with sberpm

The __`sberpm`__ python library is designed to analyze business processes and solve related problems using machine learning and process mining techniques.

# Table of Contents

[DataHolder](#DataHolder)

I. [Miners and graph visualization](#I.-Miners-and-graph-visualization)
 1. [SimpleMiner](#1.-SimpleMiner)
 2. [CausalMiner](#2.-CausalMiner)
 3. [HeuMiner](#3.-HeuMiner)
 4. [AlphaMiner](#4.-AlphaMiner)
 5. [AlphaPlusMiner](#5.-AlphaPlusMiner)
 6. [InductiveMiner](#6.-InductiveMiner)
 
II. [Metrics](#II.-Metrics)
- [Metrics + Graphs](#Metrics-+-Graphs)

III. [BPMN](#III.-BPMN)

[Machine Learning](#Machine-Learning)

IV. [Process Vectorizing](#IV.-Process-Vectorizing)

V. [Graph Embedding](#V.-Graph-Embedding)

VI. [Clustering](#V.-Clustering)

VII. [Autoinsights](#VI.-Autoinsights)

# DataHolder

The data of the business process should be an event log which stores the information about the sequence of activities in the process. Example of an event log: $W = \{(a, b, c, d), (a, c, b, d), (a, e, d) \}$, where the activities $ a $, $ b $, $ c $, $ d $ and $ e $ are sorted by their timestamps.

`DataHolder` is the data structure that contains the event log and its metainformation. It is used as an input argument by many PM algorithms.

In order to create a `DataHolder` object, it is possible either to specify a path to a log-file in the `DataHolder`'s constructor or provide the log in the form of a pandas DataFrame. It is also obligatory to set an __id_column__ and an __activity_column__. Moreover, for most Process Mining algorithms present in the library at least one of the timestamp columns (__start_timestamp_column__, __end_timestamp_column__) and a user column (__user_column__) are needed. 

### Parameters of DataHolder 

- **data (str or pd.DataFrame)** – path to a log-file (.csv, .xls(x), .txt) or pandas.DataFrame 
- **id_column (str)** – column that represents an ID of the event trace/case
- **activity_column (str)** – column that represents an activity
- __<font color='red'>*</font>start_timestamp_column (str)__ – timestamp that represents the beginning of activity execution
- __<font color='red'>*</font>end_timestamp_column (str)__ – timestamp that represents the end of activity execution
- __user_column (str)__ – column that represents the users who performed tasks in the process
- __text_column (str)__ – column with the textual data (e.g., comments)
- __duration_column (str)__ – column that contains the activity durations (If not given, it is calculated as *end_timestamp_column - start_timestamp_column*. If only one timestamp column is given, it is calculated as a difference between the timestamp of two consecutive activities, this leads to the duration of either the first or the last activity in the event trace to be NaN).
- __duration_unit (str)__ – unit of the values in a *duration_column* (needed only if the *duration_column* is given)

- __sep (str, default=',')__ – delimiter (used only if reading the log from file)
- __encoding (str)__ – encoding (used only if reading the log from file)
- __nrows (int)__ – number of rows to read from file (used only if reading the log from file)

- __preprocess (bool, default=True)__ – preprocessing of the data (sorting, removing/filling None-values, timestamp conversion), so that the library algorithms are able to work with the data
- __time_format (str)__ – format of the timestamp columns (strongly recommended to specify this parameter for correct time conversion). Time format syntax: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)
- __num_cpu (int, default=1)__ – maximum number of processors to use (for some operations that support multiprocessing)


__<font color='red'>*</font>__ For most algorithms it is needed to specify at least one of the timestamp columns. If it is unknown whether the timestamp column of the event log represents the start or end timestamp, it can be set as a __start_timestamp_column__. It is strongly recommended to specify __time_format__ parameter for correct time conversion.

### Creation of DataHolder Object

#### – using pandas DataFrame

In [1]:
from sberpm import DataHolder
import pandas as pd

df = pd.DataFrame({
    'id_column': [1, 1, 2, 2, 3, 3],
    'activity_column': ['st1', 'st2', 'st1', 'st3', 'st1','st2'],
    'start_timestamp_column': ['10.05.2020', '10.09.2020', '10.03.2020', '10.04.2020', '10.05.2020', '10.05.2020']
})

data_holder = DataHolder(data=df, 
                         id_column='id_column', 
                         activity_column='activity_column', 
                         start_timestamp_column='start_timestamp_column', 
                         time_format='%d.%m.%Y')

#### – using file path

In [2]:
path = 'example.csv'
data_holder = DataHolder(data=path, 
                         id_column='id', 
                         activity_column='stages', 
                         start_timestamp_column='dt', 
                         user_column='users', 
                         time_format='%Y-%m-%d')

When reading a log from a file, one can specify the parameters like __sep__, __encoding__, __nrows__ in the constructor of `DataHolder` if needed.

### Attributes of DataHolder
`DataHolder` stores the names of the key columns of the event log and gives an easy access to them:
- id_column
- activity_column
- start_timestamp_column
- end_timestamp_column
- user_column
- text_column
- duration_column

Moreover, `DataHolder` stores the original preprocessed log and its version grouped by *id_column* with some other columns aggregated to tuples (calculated in a lazy way when needed by some algorithms or if one calls a specific method) in the form of pandas DataFrames:
- data
- grouped_data

### Methods of DataHolder
- __check_or_calc_duration( )__ – calculate the execution time of each activity (in seconds) if not calculated before
- __get_grouped_data(...)__ – return the data (log) grouped by *id_column* with the given columns aggregated to tuples (e.g., activity_column or start_timestamp_column)
- __get_unique_activities( )__ – return the list of unique activities
- __get_columns( )__ – return the names of the columns in the event log
- __get_text( )__ – return the name of the column with the textual data if it was specified

In [3]:
data_holder.data.head(100)

,id,stages,dt,users
0,0.0,Stage_0,2019-12-01 00:00:00+00:00,Oliver
1,0.0,Stage_3,2019-12-02 00:00:00+00:00,Liam
2,0.0,Stage_5,2019-12-05 00:00:00+00:00,Emma
3,0.0,Stage_2,2019-12-11 00:00:00+00:00,Patricia
4,0.0,Stage_7,2019-12-17 00:00:00+00:00,Callum
...,...,...,...,...
95,116.0,Stage_3,2019-12-15 00:00:00+00:00,Noah
96,116.0,Stage_9,2019-12-20 00:00:00+00:00,Barbara
97,117.0,Stage_0,2019-12-01 00:00:00+00:00,Oliver
98,117.0,Stage_1,2019-12-08 00:00:00+00:00,Thomas


In [4]:
data_holder.check_or_calc_duration()

In [5]:
data_holder.data.head(3)

,id,stages,dt,users,duration
0,0.0,Stage_0,2019-12-01 00:00:00+00:00,Oliver,86400.0
1,0.0,Stage_3,2019-12-02 00:00:00+00:00,Liam,259200.0
2,0.0,Stage_5,2019-12-05 00:00:00+00:00,Emma,518400.0


In [6]:
data_holder.get_grouped_data(data_holder.activity_column, data_holder.start_timestamp_column).head()

,id,stages,dt
0,0.0,"(Stage_0, Stage_3, Stage_5, Stage_2, Stage_7)","(2019-12-01 00:00:00+00:00, 2019-12-02 00:00:0..."
1,1.0,"(Stage_0, Stage_2, Stage_4, Stage_3, Stage_8)","(2019-12-01 00:00:00+00:00, 2019-12-06 00:00:0..."
2,10.0,"(Stage_0, Stage_2, Stage_5, Stage_7)","(2019-12-01 00:00:00+00:00, 2019-12-06 00:00:0..."
3,100.0,"(Stage_2, Stage_3, Stage_8)","(2019-12-01 00:00:00+00:00, 2019-12-09 00:00:0..."
4,101.0,"(Stage_0, Stage_2, Stage_5, Stage_3, Stage_7, ...","(2019-12-01 00:00:00+00:00, 2019-12-10 00:00:0..."


# I. Miners and graph visualization

Having the event log of a business process, one can build the process model in the form of a graph by using the algorithms called *miners* and visualize it.

In order to do this, the library provides a number of algorithms that are stored in the module __`sberpm.miners`__ and they all have the only method:
- __apply( )__ – create a graph of the process that is stored in the *graph* attribute

## 1. SimpleMiner

`SimpleMiner` displays all edges found in the log (no filtering is applied).

Using the formal Process Mining language:
> The situation when activity $X$ is directly followed by $Y$ in at least one trace in the event log ($...XY...$) is called the _follows_ relation and denoted as $X>Y$ ($Y$ follows $X$).

SimpleMiner depicts the edges between the activities $X$ and $Y$, if $X>Y$.

In [7]:
from sberpm.miners import SimpleMiner

In [8]:
# Creation of the miner object. It takes DataHolder and the parameters of the algorithm
# (this miner does not have any additional parameters).
simple_miner = SimpleMiner(data_holder)

# Calculation.
simple_miner.apply()

# Getting the result
graph = simple_miner.graph

### Visualization of the Graph
For the purpose of visualization `GraphvizPainter` from the module __`sberpm.visual`__ should be used.

In [9]:
%matplotlib inline
from sberpm.visual import GraphvizPainter

Class `GraphvizPainter` has the following methods:
- __apply(...)__ – take a graph and prepare it for being visualized
- __write_graph(...)__ – save the graph in the given format (pdf, svg, gv, png)
- __show( )__ – show the graph in Jupyter Notebook

In [10]:
painter = GraphvizPainter()
painter.apply(graph)
#painter.write_graph("causal_miner.png", format='png')
painter.show()

## 2. CausalMiner

`CausalMiner` filters the edges.
> Other types of relations derived from $X>Y$:
- _causal_ relation ($X \to Y$) – if $Х>Y$ and not $Y>X$
- _parallel_ relation ($X\parallel Y $) – if $Х>Y$ and $Y>X$
- _independent_ ($X\#Y$) – if not $X>Y$ and not $Y>X$

CausalMiner depicts the edge between $X$ and $Y$if $X\to Y$.

In [11]:
from sberpm.miners import CausalMiner

In [12]:
# Miner
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## 3. HeuMiner

`HeuMiner` is a Heuristic Miner. It removes *unimportant* edges whose *unimportance* is below the given threshold. 
The **threshold** parameter must be **between 0 and 1**. The bigger it is, the less edges are depicted (the remaining ones are considered to be more important).

Source: https://www.researchgate.net/publication/229124308_Process_Mining_with_the_Heuristics_Miner-algorithm

In [13]:
from sberpm.miners import HeuMiner

In [14]:
# Miner
heu_miner = HeuMiner(data_holder, threshold=0.8)
heu_miner.apply()
graph = heu_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## 4. AlphaMiner

`AlphaMiner` takes into consideration causal, parallel and independent relations and produces a Petri net that contains the information about parallel and XOR-like executions of the activities.

In [15]:
from sberpm.miners import AlphaMiner

In [16]:
# Miner
alpha_miner = AlphaMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## 5. AlphaPlusMiner

`AlphaPlusMiner` is an implementation of Alpha+ miner. It is a modification of Alpha Miner that can handle one-loop cases (logs where $X\to X$ relation is present).

In [17]:
from sberpm.miners import AlphaPlusMiner

In [18]:
# Miner
alpha_miner_plus = AlphaPlusMiner(data_holder)
alpha_miner_plus.apply()
graph = alpha_miner_plus.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## 6. InductiveMiner

`InductiveMiner` creates a Process Tree structure. Its leaf nodes are real activities of the process and all the other nodes are the operators. There are four types of operators: 
- SEQUENTIAL (`->`), 
- EXCLUSIVE_CHOICE/OR (`X`), 
- PARALLEL/AND (`||`), 
- LOOP (`*`).

There is an additional 'operator' that sais that it is impossible to detect an appropriate operator:
- FLOWER_MODEL ('`?`')

*Note*: some of the leaf nodes might be *hidden activities* represented by black rectangles. They are not real activities and exist only to preserve the correct process tree structure.

For example, a log containing two event traces $W = \{(a, b, c), (a, c)\}$ can produce following the Process Tree: 
`SEQUENTIAL(a, EXCLUSIVE_CHOICE(b, hidden_activity), c)`.

If during an iteration the algorithm cannot find an appropriate cut of the graph corresponding to one of the operators, it is possible to do a trick: if there exists such activity, so that the graph without is has an appropriate cut, the algorithm returns the following process tree: `PARALLEL(EXCLUSIVE_CHOICE(act, hidden_activity), graph_without_act)` - roughly speaking, the activity is considered to be parallel to the graph.


This additional check can be turned on or off by **parallel_activity** argument of `InductiveMiner` class.

In [19]:
from sberpm.miners import InductiveMiner

In [20]:
# Miner
inductive_miner = InductiveMiner(data_holder)
inductive_miner.apply()
graph = inductive_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

# II. Metrics

A module __`sberpm.metrics`__ offers five basic types of metrics:
1. `ActivityMetric` – metrics by activities (group by activity_column)
2. `TransitionMetric` – metrics by transitions (two consecutive activities) (group by unique transitions)
3. `IdMetric`– metrics by ids (group by id_column)
4. `TraceMetric` – metrics by event traces (group by unique traces)
5. `UserMetric` – metrics by users (group by user_column)

In [21]:
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric

Parameters:
- __data_holder__ – object of type DataHolder to calculate metrics for
- __time_unit__ – unit of time, by default time metrics are calculated in days
- __round__ – number of digits in float numbers after decimal point

Methods for all classes:
- __apply(...)__ – calculate all available metrics
- __calc_metrics(...)__ – calculate given metrics (correspond to method names/column names in DataFrame after apply())
- __calculate_time_metrics(...)__ – calculate time metrics


- __total_duration__ – calculate the total duration
- __min_duration__ – calculate the minimum duration
- __max_duration__ – calculate the maximum duration
- __mean_duration__ – calculate the mean duration
- __median_duration__ – calculate the median duration
- __std_duration__ – calculate the standard deviation of duration
- __var_duration__ – calculate the variance of duration



Extra methods:
- ActivityMetric
    - __count__ - number of occurrences in the event log
    - __unique_ids__ - unique IDs corresponding to this object
    - __unique_ids_num__ - number of unique IDs corresponding to this object
    - __aver_count_in_trace__ - average number of occurrences of this object in the event trace
    - __loop_percent__ - loop percent (from 0 to 100)
    - __throughput__ - number of execution of this object per time unit
    - __unique_users__ - unique users working on this object
    - __unique_users_num__ - number of unique users working on this object
    - __success_rate(...)__ - id rate, that end successfully (end with 'successfull' activities)
    - __failure_rate(...)__ - id rate, that end unsuccessfully (end with 'unsuccessfull' activities)



- IdMetric
    - __trace__ - event trace (list of activities)
    - __trace_length__ - length of trace (number of activities)
    - __unique_activities__ - unique activities in trace
    - __unique_activities_num__ - number of unique activities in trace
    - __loop_percent__ - loop percent (from 0 to 100)
    - __unique_users__ - unique users working on this object
    - __unique_users_num__ - number of unique users working on this object



- TraceMetric
    - __count__ - number of occurrences in the event log
    - __ids__ - (unique) IDs corresponding to this object
    - __trace_length__ - length of trace (number of activities)
    - __unique_activities__ - unique activities in trace
    - __unique_activities_num__ - number of unique activities in trace
    - __loop_percent__ - loop percent (from 0 to 100)
    - __unique_users__ - unique users working on this object
    - __unique_users_num__ - number of unique users working on this object
    
    
- TransitionMetric
    - __count__ - number of occurrences in the event log
    - __unique_ids__ - unique IDs corresponding to this object
    - __unique_ids_num__ - number of unique IDs corresponding to this object
    - __aver_count_in_trace__ - average number of occurrences of this object in the event trace
    - __loop_percent__ - loop percent (from 0 to 100)
    - __throughput__ - number of execution of this object per time unit
    - __unique_users__ - unique users working on this object
    - __unique_users_num__ - number of unique users working on this object
    - __success_rate(...)__ - id rate, that end successfully (end with 'successfull' activities)
    - __failure_rate(...)__ - id rate, that end unsuccessfully (end with 'unsuccessfull' activities)


- UserMetric
    - __count__ - number of occurrences in the event log
    - __unique_activities__ - unique activities that user worked on
    - __unique_activities_num__ - number of unique activities that user worked on
    - __unique_ids__ - unique IDs corresponding to this object
    - __unique_ids_num__ - number of unique IDs corresponding to this object
    - __throughput__ - number of activity execution per time unit
    - __workload__ - rate of all activities in the event log executed by user


### 1. ActivityMetric

In [22]:
activity_metric = ActivityMetric(data_holder, time_unit='day')
activity_metric.apply().head()

,count,unique_ids,unique_ids_num,aver_count_in_trace,loop_percent,throughput,unique_users,unique_users_num,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
Stage_0,758,"{127.0, 153.0, 698.0, 958.0, 968.0, 686.0, 344...",758,1.0,0.0,0.200053,"{Oliver, Jake}",2,3789.0,5.079088,5.0,9.0,1.0,568935.172394,2.566106
Stage_2,672,"{127.0, 153.0, 884.0, 958.0, 968.0, 920.0, 344...",672,1.0,0.0,0.203575,"{Patricia, Mason, Olivia}",3,3301.0,5.070661,5.0,9.0,1.0,556191.654102,2.537204
Stage_8,633,"{153.0, 698.0, 884.0, 958.0, 686.0, 817.0, 114...",633,1.0,0.0,0.595484,"{Mary, Isabella}",2,1063.0,5.288557,6.0,9.0,1.0,555910.358654,2.536562
Stage_5,613,"{127.0, 698.0, 958.0, 968.0, 686.0, 344.0, 114...",613,1.0,0.0,0.245396,"{Emma, Margaret}",2,2498.0,4.976096,5.0,9.0,1.0,580655.011825,2.592401
Stage_3,577,"{127.0, 968.0, 686.0, 120.0, 741.0, 569.0, 445...",577,1.0,0.0,0.211821,"{Noah, Liam, Amelia}",3,2724.0,5.082090,5.0,9.0,1.0,573913.299176,2.577308


### 2. TransitionMetric

In [23]:
transition_metric = TransitionMetric(data_holder, time_unit='day')
transition_metric.apply().head()

,count,unique_ids,unique_ids_num,aver_count_in_trace,loop_percent,throughput,unique_users,unique_users_num,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
"(Stage_0, Stage_2)",313,"{153.0, 958.0, 968.0, 920.0, 344.0, 120.0, 520...",313,1.0,0.0,0.193210,"{Jake, Oliver}",2,1620.0,5.175719,5.0,9.0,1.0,572320.076759,2.573728
"(Stage_2, Stage_3)",242,"{499.0, 30.0, 714.0, 968.0, 906.0, 266.0, 120....",242,1.0,0.0,0.200165,"{Patricia, Mason, Olivia}",3,1209.0,4.995868,5.0,9.0,1.0,526610.094939,2.468810
"(Stage_7, Stage_8)",219,"{50.0, 504.0, 996.0, 153.0, 884.0, 958.0, 686....",219,1.0,0.0,0.198370,"{Harry, Callum}",2,1104.0,5.041096,5.0,9.0,1.0,605443.122537,2.647158
"(Stage_3, Stage_5)",184,"{499.0, 30.0, 686.0, 120.0, 654.0, 700.0, 438....",184,1.0,0.0,0.195329,"{Noah, Liam, Amelia}",3,942.0,5.119565,5.0,9.0,1.0,579147.448015,2.589034
"(Stage_5, Stage_7)",178,"{499.0, 996.0, 30.0, 896.0, 958.0, 21.0, 686.0...",178,1.0,0.0,0.199105,"{Emma, Margaret}",2,894.0,5.022472,5.0,9.0,1.0,544567.605100,2.510551


### 3. IdMetric

In [24]:
id_metric = IdMetric(data_holder, time_unit='day')
id_metric.apply().head()

,trace,trace_length,unique_activities,unique_activities_num,loop_percent,unique_users,unique_users_num,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
0.0,"(Stage_0, Stage_3, Stage_5, Stage_2, Stage_7)",5,"{Stage_0, Stage_2, Stage_5, Stage_7, Stage_3}",5,0.0,"{Callum, Patricia, Oliver, Emma, Liam}",5,16.0,4.000000,4.5,6.0,1.0,388800.0,2.121320
1.0,"(Stage_0, Stage_2, Stage_4, Stage_3, Stage_8)",5,"{Stage_0, Stage_8, Stage_2, Stage_3, Stage_4}",5,0.0,"{Jake, Noah, William, Olivia, Isabella}",5,15.0,3.750000,4.0,5.0,2.0,145800.0,1.299038
10.0,"(Stage_0, Stage_2, Stage_5, Stage_7)",4,"{Stage_5, Stage_0, Stage_7, Stage_2}",4,0.0,"{Harry, Oliver, Mason, Emma}",4,16.0,5.333333,5.0,8.0,3.0,364800.0,2.054805
100.0,"(Stage_2, Stage_3, Stage_8)",3,"{Stage_8, Stage_3, Stage_2}",3,0.0,"{Mason, Liam, Isabella}",3,17.0,8.500000,8.5,9.0,8.0,21600.0,0.500000
101.0,"(Stage_0, Stage_2, Stage_5, Stage_3, Stage_7, ...",7,"{Stage_0, Stage_8, Stage_2, Stage_5, Stage_7, ...",7,0.0,"{Harry, Jake, Mary, Margaret, Mason, Barbara, ...",7,26.0,4.333333,4.0,9.0,1.0,854400.0,3.144660


### 4. TraceMetric

In [25]:
trace_metric = TraceMetric(data_holder, time_unit='day')
trace_metric.apply().head()

,count,ids,trace_length,unique_activities_num,loop_percent,unique_users,unique_users_num,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
"(Stage_0, Stage_2, Stage_3, Stage_8)",12,"{276.0, 652.0, 833.0, 240.0, 832.0, 463.0, 939...",4,4,0.0,"{Jake, Noah, Mary, Patricia, Oliver, Mason, Ol...",10,196.0,16.333333,18.0,25.0,8.0,2.035200e+06,4.853407
"(Stage_0, Stage_2, Stage_3, Stage_5, Stage_8)",12,"{665.0, 351.0, 158.0, 81.0, 20.0, 936.0, 124.0...",5,5,0.0,"{Jake, Mary, Noah, Margaret, Patricia, Oliver,...",11,247.0,20.583333,20.5,32.0,12.0,2.296200e+06,5.155229
"(Stage_0, Stage_2, Stage_5)",11,"{565.0, 514.0, 645.0, 792.0, 600.0, 58.0, 73.0...",3,3,0.0,"{Jake, Margaret, Patricia, Oliver, Mason, Oliv...",7,129.0,11.727273,10.0,18.0,6.0,1.006810e+06,3.413633
"(Stage_0, Stage_2, Stage_3, Stage_5, Stage_7, Stage_8)",10,"{227.0, 361.0, 477.0, 738.0, 478.0, 790.0, 770...",6,6,0.0,"{Harry, Jake, Noah, Callum, Mary, Margaret, Pa...",14,252.0,25.200000,26.0,32.0,16.0,1.931904e+06,4.728636
"(Stage_0, Stage_2, Stage_3, Stage_5)",10,"{267.0, 78.0, 174.0, 924.0, 834.0, 225.0, 261....",4,4,0.0,"{Noah, Margaret, Patricia, Oliver, Mason, Oliv...",9,161.0,16.100000,14.5,25.0,11.0,1.649376e+06,4.369210


### 5. UserMetric

In [26]:
user_metric = UserMetric(data_holder, time_unit='day')
user_metric.apply().head()

,count,unique_activities,unique_activities_num,unique_ids,unique_ids_num,throughput,workload,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
Oliver,493,{Stage_0},1,"{127.0, 153.0, 698.0, 958.0, 686.0, 920.0, 235...",493,0.195093,0.110513,2527.0,5.199588,5.0,9.0,1.0,572380.429813,2.573864
Isabella,427,{Stage_8},1,"{153.0, 698.0, 884.0, 686.0, 817.0, 920.0, 235...",427,0.610000,0.095718,700.0,5.303030,6.0,9.0,1.0,544502.479339,2.510401
Emma,415,{Stage_5},1,"{127.0, 698.0, 958.0, 968.0, 686.0, 344.0, 114...",415,0.242974,0.093028,1708.0,4.936416,5.0,9.0,1.0,596459.941862,2.627446
Harry,328,{Stage_7},1,"{127.0, 698.0, 958.0, 557.0, 120.0, 741.0, 445...",328,0.303142,0.073526,1082.0,4.852018,5.0,9.0,1.0,617630.823061,2.673669
Jake,265,{Stage_0},1,"{171.0, 60.0, 21.0, 968.0, 344.0, 550.0, 216.0...",265,0.209984,0.059404,1262.0,4.853846,5.0,9.0,1.0,555766.721893,2.536234


### TokenReplay

`TokenReplay` allows to compute *fitness* that indicates to what extent a Petri net captures a business process (1 for good, 0 for bad). Fitness is calculated for each event trace (id) by replaying it in the given Petri net as follows: 
$$ Fitness = \frac{1}{2}\big(1-\frac{missed}{consumed}\big) + \frac{1}{2}\big(1-\frac{remaining}{produced}\big) $$
- $produced$ – the number of produced tokens
- $consumed$ – the number of consumed tokens
- $remaining$ – the number of remaining tokens
- $missing$ – the number of missing tokens

The following metrics can be obtained:
- values of the four components and fitness for each event traces
- mean fitness of all traces (__mean_fitness__)
- fitness of the log by substituting the values in the formula by sum of the corresponding values of each event trace  (__average_fitness__)

In [27]:
from sberpm.metrics import TokenReplay

In [28]:
token_replay = TokenReplay(data_holder, alpha_miner.graph)
token_replay.apply()
token_replay.result

,id,stages,c,p,m,r,fitness
0,0.0,"(Stage_0, Stage_3, Stage_5, Stage_2, Stage_7)",7,8,5,6,0.267857
1,1.0,"(Stage_0, Stage_2, Stage_4, Stage_3, Stage_8)",6,9,4,7,0.277778
2,10.0,"(Stage_0, Stage_2, Stage_5, Stage_7)",6,7,4,5,0.309524
3,100.0,"(Stage_2, Stage_3, Stage_8)",4,5,2,3,0.450000
4,101.0,"(Stage_0, Stage_2, Stage_5, Stage_3, Stage_7, ...",14,10,10,6,0.342857
...,...,...,...,...,...,...,...
994,995.0,"(Stage_0, Stage_2, Stage_5)",4,6,2,4,0.416667
995,996.0,"(Stage_3, Stage_2, Stage_5, Stage_7, Stage_8)",7,7,5,5,0.285714
996,997.0,"(Stage_0, Stage_2, Stage_3, Stage_5, Stage_6, ...",10,11,8,9,0.190909
997,998.0,"(Stage_0, Stage_1, Stage_2, Stage_3, Stage_9)",11,11,6,6,0.454545


In [29]:
print('mean:', token_replay.mean_fitness)
print('average:', token_replay.average_fitness)

mean: 0.36138350149788057
average: 0.3365537369071906


# Metrics + Graphs

In _sberpm_ there is an opportunity to visualize metrics on the graph. In order to do it, one can use `Graph` class that has the following methods:
- __add_node_metric(...)__ – add metric to the graph node
- __add_edge_metric(...)__ – add metric to the graph edge

In [30]:
# Calculation of the metrics
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Getting the graph
graph = causal_miner.graph

# Adding the metrics to the graph
graph.add_node_metric('count', nodes_count_metric)
graph.add_edge_metric('count', edges_count_metric)
graph.add_node_metric('mean_time', mean_time_node_metric)

In [31]:
painter = GraphvizPainter()

# Specify which metrics influence the nodes' colors and the edges' widths
painter.apply(graph, node_style_metric='count', edge_style_metric='count')
# or 
# painter.apply(graph, node_style_metric='mean_time', edge_style_metric='count')

painter.write_graph("metric_graph.png", format = 'png')
painter.show()

To delete metrics from the graph, the following methods should be used:
- __clear_node_metrics(...)__ – delete all metrics from nodes
- __clear_edge_metrics__ – delete all metrics from edges

In [32]:
graph.clear_node_metrics()
graph.clear_edge_metrics()

# III. BPMN

In order to save process graph in a BPMN (Business Process Model and Notation) format, one can use `BpmnExporter` from __`sberpm.bpmn`__ module. It has the following methods:
- __apply_petri(...)__ – construct BPMN for a Petri net 
- __get_string_representation()__ – get BPMN notation of the graph
- __write(...)__ – write graph in a BPMN format to file

At this moment, only Petri nets can be converted to BPMN.

In [33]:
from sberpm.bpmn import BpmnExporter

In [34]:
bpmn_exporter = BpmnExporter()
bpmn_exporter.apply_petri(alpha_miner.graph)
bpmn_exporter.get_string_representation()[:1000]

d:\sber\lib\sber_process_mining\sberpm\bpmn\_bpmn_graph_to_file\_bpmn_exporter.py:63: RuntimeWarning: Impossible to create orthogonal edges, splines will be created instead.
  warnings.warn("Impossible to create orthogonal edges, splines will be created instead.", RuntimeWarning)


'<bpmn:definitions id="Definitions_123" targetNamespace="http://bpmn.io/schema/bpmn" xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI">\n\t<bpmn:process id="Process_123">\n\t\t<bpmn:task bpmn:incoming="sequenceFlow_15" bpmn:outgoing="sequenceFlow_0" id="task_0" name="Stage_0" />\n\t<bpmn:parallelGateway bpmn:incoming="sequenceFlow_0" bpmn:outgoing="sequenceFlow_11" id="parallelGateway_0" />\n\t<bpmn:task bpmn:incoming="sequenceFlow_16" bpmn:outgoing="sequenceFlow_1" id="task_1" name="Stage_3" />\n\t<bpmn:exclusiveGateway bpmn:incoming="sequenceFlow_7" bpmn:outgoing="sequenceFlow_22" id="exclusiveGateway_0" />\n\t<bpmn:task bpmn:incoming="sequenceFlow_17" bpmn:outgoing="sequenceFlow_2" id="task_2" name="Stage_5" />\n\t<bpmn:task bpmn:incoming="sequenceFlow_18" bpmn:outgoing="sequenceFlow_3" id="task_3" name="Stage_2" />\n\t<bp

In [35]:
bpmn_exporter.write('exported.bpmn')

To load BPMN file, class `BpmnImporter` with the following methods can be used:
- __load_bpmn_from_xml__ – load BPMN graph
- __get_pydotplus_graph__ – get graph in a pydotplus format

In [36]:
from sberpm.bpmn import BpmnImporter

In [37]:
bpmn_importer = BpmnImporter()
bpmn_importer.load_bpmn_from_xml('exported.bpmn')
pydot_graph = bpmn_importer.get_pydotplus_graph()
pydot_graph.write('imported_bpmn.svg', prog='dot', format='svg')

True

# Machine Learning

A module for machine learning __`sberpm.ml`__ allows for process vectorization and clustering. New analytical instruments will be available later.

## IV. Process Vectorizing

For process vectorization the class `ProcessCountVectorizer` from module __`sberpm.ml.vectorizer`__ can be used. It transforms  each event trace to a corresponding vector.

In [38]:
from sberpm.ml.vectorizer import ProcessCountVectorizer

In [39]:
vectorizer = ProcessCountVectorizer()
embeddings = vectorizer.transform(data_holder)

In [40]:
embeddings.shape

(999, 10)

In [41]:
embeddings

array([[1, 0, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 1, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 1],
       [1, 0, 1, ..., 1, 0, 0]])

Methods of `ProcessCountVectorizer`:
- __transform(...)__ – return vectors for each event trace in the given DataHolder

## V. Graph Embedding

Graphs created by miners can be embedded (reduced to lower dimensions) with the help of the class __`GraphEmbedder`__ from module __`sberpm.ml.vectorizer`__. It transforms a data holder's graph built with an input miner to dimension k using one of the 5 high-order proximity functions between vertices.

In [42]:
from sberpm import DataHolder
from sberpm.miners import SimpleMiner
from sberpm.ml.vectorizer import GraphEmbedder
from sberpm.ml.vectorizer import katz_index, rooted_pr, adamic_adar, common_neighbors, vertex_similarity

In [43]:
path = 'example.csv'
data_holder = DataHolder(data=path, 
                         id_column='id', 
                         activity_column='stages', 
                         start_timestamp_column='dt', 
                         user_column='users', 
                         time_format='%Y-%m-%d')

In [44]:
graph_vectorizer = GraphEmbedder()
graph_embeddings = graph_vectorizer.transform(data_holder, SimpleMiner, 6, katz_index, 3)

In [45]:
# source vertices embedding vectors 
print(graph_embeddings[0])

[[ 3.07202172e-01 -3.71173613e-01  3.50081222e-01  3.71565006e-01
   1.80074702e-01 -1.14576300e+00 -1.12446411e-01 -9.61481343e-17
   2.57535228e-16 -1.73533797e-01  0.00000000e+00  0.00000000e+00]
 [-4.39460335e-01  2.61643967e-02  4.75337814e-01  2.18259908e-01
   3.07809306e-02  6.33823991e-01 -4.96635418e-01 -4.44685121e-16
   8.36989491e-17 -2.96628661e-02  2.20738597e-16  2.26570500e-16]
 [ 2.63553041e-01  5.82155227e-02  6.23727045e-02  2.91545820e-01
  -5.45412047e-01  6.21055672e-01  5.82155227e-02  2.28282728e-01
  -4.55544545e-01 -2.98453844e-01  4.43464552e-03 -4.53141000e-16]
 [ 1.03724212e-02 -3.82947639e-01 -4.10294004e-01  1.14741079e-02
  -1.69230888e-01  2.44423323e-02 -3.82947639e-01  4.40638504e-01
   1.16376528e-01  2.72011927e-01  1.68596247e-01 -6.79711500e-16]
 [-2.71277050e-01 -2.03404445e-01 -2.17929596e-01 -3.00090219e-01
  -2.92045534e-01 -6.39257092e-01 -2.03404445e-01 -3.72996400e-01
  -3.19965384e-01 -3.56519044e-01  1.03933266e-01 -2.26570500e-16]
 [-6.

In [46]:
# target vertices embedding vectors
print(graph_embeddings[1])

[[-3.07202172e-01  3.71173613e-01 -3.50081222e-01 -3.71565006e-01
  -1.80074702e-01  1.14576300e+00  1.12446411e-01  1.34269315e-16
  -3.04093910e-16  1.73533797e-01  0.00000000e+00 -3.00503926e-17]
 [ 4.39460335e-01 -2.61643967e-02 -4.75337814e-01 -2.18259908e-01
  -3.07809306e-02 -6.33823991e-01  4.96635418e-01  3.66462245e-16
  -2.70879027e-16  2.96628661e-02  0.00000000e+00 -6.51068092e-16]
 [-2.76936315e-01 -4.12253111e-02 -4.41692186e-02 -3.06350572e-01
   4.86022862e-01 -6.52592998e-01 -4.12253111e-02 -1.89377825e-01
   5.50358249e-01  2.07622322e-01  0.00000000e+00  5.77165518e-02]
 [-7.52072174e-02  4.06101439e-01  4.35101221e-01 -8.31952073e-02
   2.45233821e-01 -1.77223791e-01  4.06101439e-01 -2.66923963e-01
  -2.09389391e-01 -2.96103805e-01  0.00000000e+00 -2.53195146e-01]
 [ 2.87585964e-01  1.98150583e-01  2.12300555e-01  3.18131354e-01
   2.71896847e-01  6.77688611e-01  1.98150583e-01  3.26854097e-01
   3.46752585e-01  3.60955249e-01  0.00000000e+00 -2.34942454e-01]
 [ 2.

In [47]:
nodes = graph_vectorizer.get_vtx2nodes()
# embeddings[0][2] is the embedding of vertex 2
print(nodes[1])
vertices_from_nodes = graph_vectorizer.get_node_obj2vtx()
# gives mapping from node objects to vertices
print(vertices_from_nodes[nodes[1]])
vertices_from_nodeids = graph_vectorizer.get_node_id2vtx()
print(vertices_from_nodeids[nodes[1].id])

Node: Stage_3
1
1


It is possible to vectorize event traces based on the graph embedding above with the help of __`HopeVectorizer`__ module. It works similarly to GraphEmbedder.

In [48]:
from sberpm import DataHolder
from sberpm.miners import SimpleMiner
from sberpm.ml.vectorizer import HopeVectorizer
from sberpm.ml.vectorizer import katz_index, rooted_pr, adamic_adar, common_neighbors, vertex_similarity


In [49]:
hope_vectorizer = HopeVectorizer()
hope_embeddings = hope_vectorizer.transform(data_holder, SimpleMiner, 7, katz_index, 2.5)

In [50]:
failed_to_vectorize_traces = hope_embeddings[1]
print(failed_to_vectorize_traces)

[('Stage_9',)]


In [51]:
trace_vectors = hope_embeddings[0]
trace_vector = trace_vectors[0][0]
trace = trace_vectors[0][1]
print(trace_vector)
print(trace)

[[-0.14742121 -0.02001389 -0.31527061  0.68472939 -0.05691703 -0.05289525
  -0.15972771  0.25353948 -0.05665196 -0.00968967 -0.12284206  0.2626642
  -0.14742121 -0.02001389 -0.31527061  0.68472939]]
('Stage_0', 'Stage_2', 'Stage_3', 'Stage_5', 'Stage_7')


## VI. Clustering

After the process is vectorized, the obtained vectors can be clustered with `GraphClustering` from __`sberpm.ml.processes`__.

In [52]:
from sberpm.ml.processes import GraphClustering

Methods of `GraphClustering`:
- __fit(...)__ – compute clustering
- __predict(...)__ – predict cluster index for each event trace
- __add_cluster_column(...)__ – add column with cluster index to the grouped data
- __predict_add(...)__ – compute clustering and add predicted cluster index to the grouped data

In [53]:
estimator = GraphClustering()
estimator.fit(embeddings)
clusters = estimator.predict(embeddings)

In [54]:
clusters.shape

(999,)

In [55]:
clusters[:10]

array([0, 1, 0, 1, 0, 1, 0, 1, 1, 1])

## VII. Autoinsights

An autoinsight module __`sberpm.autoinsights`__ provides means to automatically detect process problems such as bottlenecks and deviations and map them on the process graph. By using the class `AutoInsights`, the following factors are taken into account:
1. Transition duration and workload
2. Utilization rate of transition
3. Process loops and repetitions

`AutoInsights` contains the following parameters:
- __data_holder__ – pandas.DataFrame or sberpm.DataHolder
- __time_unit__ – unit of time duration values, by default time metrics are calculated in _days_

In [56]:
from sberpm.autoinsights import AutoInsights

In [57]:
# Initialization
auto_i = AutoInsights(data_holder, time_unit='day')

`AutoInsights` has the following methods:
- __apply(...)__ – find inefficiencies in the input log
- __get_graph()__ – return process graph with discovered autoinsights, the type depends on the __mode__ parameter in the __apply(...)__ method
- __describe_nodes()__ – return the table of activities with the autoinsight metric indicator
- __describe_edges()__ – return the table of transitions with the autoinsight metric indicator

Parameters of the __apply(...)__ method:
- __miner__ – miner to use to search for inefficiencies
- __mode__ – autoinsight search mode, can be either *time*, *cycles* or *overall*, by default *overall*
- __width_by_insight__ – if True, the edges' width on the graph will correspond to the "insight" status
- __q_min__ – the lower quantile for the algorithm, by default 0.1
- __q_top__ – the upper quantile for the algorithm, by default 0.85

Other methods do not require any parameters.

### Autoinsight Visualization

Three ways of visualizations are supported at the moment:
- By transition duration __(mode='time')__ – the longest-lasting transitions are colored red
- By cycling __(mode='cycles')__ – transitions with returns are colored blue, the loops themselves are not reflected
- Hybrid method __(mode='overall')__ – all transitions are given a status (one of three):
    -  __Optimal__ (positive insights)
    - __<font color='grey'>No insight</font>__
    - __<font color='red'>Need for optimization</font>__ (negative insights)
    
The mean duration and the return rate for both optimal insights and the ones demanding optimization are displayed in the corner of the image. 


In [58]:
# Transition duration
auto_i.apply(miner=simple_miner, mode='time')
graph = auto_i.get_graph()

painter = GraphvizPainter()
painter.apply_insights(graph)
painter.show()

In [59]:
# Loops
auto_i.apply(miner=simple_miner, mode='cycles')
graph = auto_i.get_graph()

painter = GraphvizPainter()
painter.apply_insights(graph)
painter.show()

In [60]:
# Hybrid mode
auto_i.apply(miner=simple_miner, mode='overall')
graph = auto_i.get_graph()

painter = GraphvizPainter()
painter.apply_insights(graph)
painter.show()

### Description of the Results

The results are set out in a table format. For each element insight and metric indicators are given: 
- __1__ – negative insight
- __0__ – not insight
- __-1__ – positive insight

In [61]:
auto_i.describe_nodes().head()

,activities,count,mean_duration,loop_percent,aver_count_in_trace,throughput,unique_users_num,user_value,insights
0,Stage_0,1,0,0,0,-1,0,1,1
1,Stage_1,0,-1,0,0,0,0,-1,-1
2,Stage_2,1,0,0,0,0,0,0,1
3,Stage_3,0,0,0,0,0,0,0,0
4,Stage_4,0,0,0,0,0,0,0,0


In [62]:
auto_i.describe_edges().head()

,transitions,count,mean_duration,loop_percent,aver_count_in_trace,throughput,unique_users_num,user_value,insights
0,"(Stage_0, Stage_1)",1,0,0,0,0,0,0,1
1,"(Stage_0, Stage_2)",1,0,0,0,0,0,0,1
2,"(Stage_0, Stage_3)",1,0,0,0,0,0,0,1
3,"(Stage_0, Stage_4)",0,0,0,0,1,0,0,1
4,"(Stage_0, Stage_5)",0,0,0,0,0,0,0,0
